# Video Summarizer 모델 학습/평가 코드

In [1]:
from data_loader import DataLoader
from model import c3d_model as YasuoNet
from trainer import Trainer
from tensorflow.keras.optimizers import Adam
import numpy as np
import math

## 하이퍼파라미터 설정

In [2]:
dataset_dir = r'E:\Work\YasuoNet\data\dataset14_sl5_vsr3_vw64_vh64_asr11025'
ckpt_dir = 'checkpoints'
learning_rate = 1e-4
epochs = 3
batch_size = 500

## 데이터 로더 생성

In [3]:
data_loader = DataLoader(dataset_dir)
data_config = data_loader.get_metadata()['config']
video_frames = int(data_config['segment_length'] * data_config['video_sample_rate'])
video_width = data_config['video_width']
video_height = data_config['video_height']
video_channels = 3  # 향후 메타데이터에서 읽어오도록 수정

# epoch 당 배치 수
train_steps = data_loader.get_train_data_count() // batch_size
valid_steps = math.ceil(data_loader.get_valid_data_count() / batch_size)
test_steps = math.ceil(data_loader.get_test_data_count() / batch_size)
train_steps, valid_steps, test_steps

(14, 5, 5)

## 모델 생성

In [4]:
model = YasuoNet(video_frames, video_width, video_height, video_channels)
model.compile(Adam(learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

## 모델 학습

In [5]:
trainer = Trainer(model, ckpt_dir, learning_rate, epochs)
trainer.train(
    data_loader.iter_train_batch_data(batch_size), train_steps,
    data_loader.iter_valid_batch_data(batch_size), valid_steps
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 14 steps, validate for 5 steps
Epoch 1/3
13/14 [==========================>...] - ETA: 27s - loss: 0.6722 - accuracy: 0.8863
Epoch 00001: val_loss improved from inf to 0.55933, saving model to checkpoints\ckpt-0001-0.5593.hdf5
14/14 [==============================] - 403s 29s/step - loss: 0.6684 - accuracy: 0.8880 - val_loss: 0.5593 - val_accuracy: 0.9237
Epoch 2/3
13/14 [==========================>...] - ETA: 27s - loss: 0.5763 - accuracy: 0.9100
Epoch 00002: val_loss improved from 0.55933 to 0.49862, saving model to checkpoints\ckpt-0002-0.4986.hdf5
14/14 [==============================] - 404s 29s/step - loss: 0.5749 - accuracy: 0.9100 - val_loss: 0.4986 - val_accuracy: 0.9237
Epoch 3/3
13/14 [==========================>...] - ETA: 27s - loss: 0.5428 - accuracy: 0.9100
Epoch 00003: val_loss improved from 0.49862 to 0.47307, saving model to checkpoints\ckpt-0003-0.4731.hdf5
14/14 [==============================] - 404s 29s/s

In [6]:
trainer.test(data_loader.iter_test_batch_data(batch_size), test_steps)

  ...
    to  
  ['...']
5/5 [==============================] - 12s 2s/step - loss: 0.5232 - accuracy: 0.8904


In [38]:
test_batch_iter = data_loader.iter_test_batch_data(batch_size, repeat=False)

result = []
for x, y in test_batch_iter:
    pred = model.predict(x)
    y_hat = (pred > 0.5) * 1

    result.append(np.vstack([y.squeeze(), y_hat.squeeze()]).T)
    
result = np.vstack(result)

In [39]:
len(result)

2227

In [40]:
result.sum(axis=0)

array([244,   0])